# Singapore Flat Resale price Prediction

## Step 1- Reading the data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import joblib

In [3]:
df = pd.read_csv('ResaleFlatPricesBasedonApprovalDate1990 to 1999_1.csv')
df1 = pd.read_csv('ResaleFlatPricesBasedonApprovalDate2000 to Feb2012_2.csv')
df2 = pd.read_csv('ResaleFlatPricesBasedonRegistrationDateFromMar2012 to Dec2014_3.csv')
df3 = pd.read_csv('ResaleFlatPricesBasedonRegistrationDateFromJan2015 to Dec2016_4.csv')
df4 = pd.read_csv('ResaleflatpricesbasedonregistrationdatefromJan2017onwards_5.csv')

In [4]:
df_combined=pd.concat([df,df1,df2,df3,df4], axis=0, ignore_index= True)

## Step 2 -Data Cleaning

In [ ]:
df_combined.head()

In [ ]:
df_combined.info()

In [ ]:
df_combined.isnull().sum()

In [8]:
df_combined['month']= pd.to_datetime(df_combined['month'],format='%Y-%m')

In [9]:
df_combined['remaining_lease'] = 99-(df_combined['month'].dt.year - df_combined['lease_commence_date'])

In [ ]:
df_combined.describe()

In [11]:
#remaining lease cannot be more than 99
df1_combined = df_combined[df_combined['remaining_lease'] <= 99].reset_index(drop=True)

In [ ]:
df1_combined.describe()

In [ ]:
df1_combined.shape

In [ ]:
df1_combined['mid_storey']= (df1_combined['storey_range'].str.split('TO').str[0].astype(int) + 1)
df1_combined.head()

In [ ]:
df1_combined.columns

In [ ]:
train_df = df1_combined.iloc[:int(df1_combined.shape[0] * 0.9)]
train_df.head()

## Step 3 - EDA

In [ ]:
train_df['year']=train_df['month'].dt.year

In [18]:
categorical_columns = train_df.select_dtypes(include ='object').columns
numerical_columns = train_df.select_dtypes(include=np.number).columns

In [ ]:
categorical_columns

In [20]:
numerical_columns=[ 'year','floor_area_sqm','remaining_lease', 'mid_storey','resale_price']

In [21]:
numerical_columns_1=['year','floor_area_sqm','remaining_lease','mid_storey']

In [ ]:
for i in numerical_columns:
    plt.figure(figsize=(12, 6))
    # Plot before log transformation
    sns.histplot(data=train_df,x=i,bins=50,kde=True)
    plt.title(f'{i}')

In [ ]:
train_df.head()

In [ ]:
for i in numerical_columns_1:
    fig=px.scatter(train_df, x=i, y='resale_price', trendline='ols', trendline_color_override= 'red')
    fig.show()


In [ ]:
sns.lmplot(data=train_df, x= 'year', y = 'resale_price', hue='flat_type')
plt.show()

In [ ]:
sns.lmplot(data=train_df, x= 'year', y = 'resale_price', hue='town')
plt.show()

In [23]:
df1=train_df.copy()

In [24]:
df1['remaining_lease_log']=np.log(df1['remaining_lease'])
df1['floor_area_sqm_log']=np.log(df1['floor_area_sqm'])
df1['mid_storey_log']=np.log(df1['mid_storey'])
df1['year_log']=np.log(df1['year'])
df1['resale_price_log']=np.log(df1['resale_price'])

In [25]:
numerical_columns_logged=['year_log','floor_area_sqm_log','remaining_lease_log','mid_storey_log','resale_price_log']

In [26]:
numerical_columns_logged_1=['year_log','floor_area_sqm_log','remaining_lease_log','mid_storey_log']

In [ ]:
for i,j in zip(numerical_columns,numerical_columns_logged):
    plt.figure(figsize=(12, 6))
    # Plot before log transformation
    plt.subplot(1, 2, 1)
    sns.histplot(data=df1,x=i,bins=50,kde=True)
    plt.title(f'before log {i}')

    # Plot after log transformation
    plt.subplot(1, 2, 2)
    sns.histplot(data=df1,x=j,bins=50,kde=True)
    plt.title(f'after log-Transformed {j}')
    # Display the plots
    plt.tight_layout()
    plt.show()

# Step 4 - Feature Engineering and Feature selection

# Feature selection

In [ ]:
df1.columns

Features like month, block, street_name, storey_range,lease_commence_Date, are not so important for resale flat price predicton and so they are dropped

Features like floor_area_sqm, remaining_lease, mid_storey are dropped because we dont need those as we have logged values of those features

In [28]:
selected_features=['town','flat_type','year_log','remaining_lease_log','floor_area_sqm_log', 'mid_storey_log']

# Feature Engineering

In [29]:
df2 = df1[['town', 'flat_type','year_log','remaining_lease_log','floor_area_sqm_log', 'mid_storey_log','resale_price_log']]

In [ ]:
numerical_columns_1

In [ ]:
numerical_columns_logged_1

## Step 5 - Handling outliers

In [ ]:
#Detecting Outliers before and after log transformation
for i,j in zip(numerical_columns_1,numerical_columns_logged_1):
        plt.figure(figsize=(10, 6))
    # Plot before log transformation
        plt.subplot(1, 2, 1)
        sns.boxplot(data=train_df, x=i)
        plt.title(f"Boxplot to Detect Outliers before log of {i}")
        
        
        plt.subplot(1, 2, 2)
        sns.boxplot(data=df2, x=j)
        plt.title(f"Boxplot to Detect Outliers after log of {j}")
        plt.tight_layout()
        plt.show()

In [39]:
#There are some outliers in floor_area_sqm_log, mid_storey_log,remaining_lease_log

In [32]:
def outlier(df, column):
    iqr = df[column].quantile(0.75) - df[column].quantile(0.25)
    upper = df[column].quantile(0.75) + (1.5*iqr)
    lower = df[column].quantile(0.25) - (1.5*iqr)
    df_filtered = df[(df[column] >= lower) & (df[column] <= upper)]
    return df_filtered

In [ ]:
df3 = outlier(df2,'floor_area_sqm_log').reset_index(drop=True)
df3

In [ ]:
df4=outlier(df3,'mid_storey_log').reset_index(drop=True)
df4

In [ ]:
df5=outlier(df4,'remaining_lease_log').reset_index(drop=True)
df5

In [ ]:
for i in numerical_columns_logged_1:
        plt.figure(figsize=(10, 6))
        plt.subplot(1, 2, 1)
        sns.boxplot(data=df2, x=i)
        plt.title(f"Boxplot to Detect Outliers before Outlier removal of {i}")

        plt.subplot(1, 2, 2)
        sns.boxplot(data=df5, x=i)
        plt.title(f"Boxplot to Detect Outliers after Outlier_removal of {i}")
        plt.tight_layout()
        plt.show()

In [ ]:
df5.columns

## Step 6 - Encoding

In [37]:
df6=df5.copy()

In [38]:
df6['flat_type']=df6['flat_type'].str.replace('-', ' ')

In [ ]:
df6['flat_type'].unique()

In [40]:
flat_type_map={'1 ROOM' :1 , '2 ROOM': 2, '3 ROOM': 3, '4 ROOM': 4, '5 ROOM': 5,  'EXECUTIVE':6,
       'MULTI GENERATION': 7}

df6['flat_type_encoded']=df6['flat_type'].map(flat_type_map)

In [41]:
town_encoded={'ANG MO KIO':1, 'BEDOK':2, 'BISHAN':3, 'BUKIT BATOK':4, 'BUKIT MERAH':5,
       'BUKIT TIMAH':6, 'CENTRAL AREA':7, 'CLEMENTI':8, 'GEYLANG':9, 'HOUGANG':10,
       'JURONG EAST':11, 'JURONG WEST':12, 'KALLANG/WHAMPOA':13, 'MARINE PARADE':14,
       'QUEENSTOWN':15, 'SERANGOON':16, 'TAMPINES':17, 'TOA PAYOH':18, 'WOODLANDS':19,
       'YISHUN':20, 'CHOA CHU KANG':21, 'BUKIT PANJANG':22, 'PASIR RIS':23,
       'SENGKANG':24, 'SEMBAWANG':25, 'LIM CHU KANG':26, 'PUNGGOL':27}
df6['town_encoded']=df6['town'].map(town_encoded)

In [42]:
df7=df6[['town_encoded','flat_type_encoded','year_log','floor_area_sqm_log', 'remaining_lease_log','mid_storey_log']]

## Step 7 - Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler_target=StandardScaler()
df8_scaled=scaler.fit_transform(df7)
df8_target_scaled=scaler_target.fit_transform(df6[['resale_price_log']])
df8_scaled

In [ ]:
df8_scaled=pd.DataFrame(df8_scaled, columns=['town_encoded','flat_type_encoded','year_log','floor_area_sqm_log', 'remaining_lease_log','mid_storey_log'])
df8_scaled.head()

In [ ]:
df8_scaled_target=pd.DataFrame(df8_target_scaled, columns=['resale_price_log'])
df8_scaled_target.head()

In [46]:
df8_scaled_1=pd.concat([df8_scaled,df8_scaled_target],axis=1)

In [47]:
selected_features_AS = ['town_encoded','flat_type_encoded','year_log','floor_area_sqm_log', 'remaining_lease_log','mid_storey_log']

## Step 8 - Model Selection

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [49]:
# Randomly sample 20% of your dataset
df_sample = df8_scaled_1.sample(frac=0.2, random_state=42)
X_sample = df_sample.drop('resale_price_log',axis=1)
y_sample = df_sample['resale_price_log']

In [ ]:
df_sample.isnull().sum()

In [51]:
from sklearn.model_selection import train_test_split
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

In [ ]:
X_train.isnull().sum()

In [53]:
def model_fit_evaluation_regr(models_reg, x_train, x_test, y_train, y_test, metrics_reg):
    results_reg = {}
    
    for model_name_reg, model_reg in models_reg.items():
        regression = model_reg
        regression.fit(x_train, y_train)
        y_prediction = regression.predict(x_test)
        results_reg[model_name_reg] = {}
        for metric_name_reg, metric_func_reg in metrics_reg.items():
            results_reg[model_name_reg][metric_name_reg] = metric_func_reg(y_test, y_prediction)
    return results_reg

models_reg = {
    'Linear Regression': LinearRegression(),
    'Ridge':Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5)}

metrics_reg = {'MSE' : mean_squared_error,
               'MAE': mean_absolute_error,
                'R2_Score' : r2_score} 

In [ ]:
model_fit_evaluation_regr(models_reg, X_train, X_test, y_train, y_test, metrics_reg)

In [55]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [56]:
def model_fit_evaluation_Non_lin_regr(models_reg_Non_lin, x_train, x_test, y_train, y_test, metrics_reg_Non_lin):
    results_reg = {}
    
    for model_name_reg, models_reg_Non_lin in models_reg_Non_lin.items():
        regression = models_reg_Non_lin()
        regression.fit(x_train, y_train)
        y_prediction_reg = regression.predict(x_test)
        results_reg[model_name_reg] = {}
        for metric_name_reg, metric_func_reg in metrics_reg_Non_lin.items():
            results_reg[model_name_reg][metric_name_reg] = metric_func_reg(y_test, y_prediction_reg)
    return results_reg

models_reg_Non_lin = {
    'Gradient Boosting Regression':GradientBoostingRegressor,
    'Random Forest Regression': RandomForestRegressor,
    'Decision Tree Regression': DecisionTreeRegressor}

metrics_reg_Non_lin = {'MSE' : mean_squared_error,
               'MAE': mean_absolute_error,
                'R2_Score' : r2_score} 

In [ ]:
model_fit_evaluation_Non_lin_regr(models_reg_Non_lin, X_train, X_test, y_train, y_test, metrics_reg_Non_lin)

# Random Forest Regressor is selected as it has higher R_Square

## Step 9 - Model Training 

In [58]:
x_model = df8_scaled_1[['town_encoded','flat_type_encoded','year_log','floor_area_sqm_log', 'remaining_lease_log', 'mid_storey_log']]  
y_model = df8_scaled_1[['resale_price_log']]
x_train_model, x_test_model, y_train_model, y_test_model = train_test_split(x_model, y_model, test_size=0.2, random_state=42)

In [ ]:
selected_model = RandomForestRegressor()

# Train the model (fit to training data)
selected_model.fit(x_train_model, y_train_model)


## Step 10 - Model Export

In [60]:
import joblib

In [64]:
model_file_name=r"D:\GUVI_projects\model_RF.joblib"

In [65]:
saved_data = {'model': selected_model,'scaler':scaler,'scaler_target':scaler_target}

In [ ]:
joblib.dump(saved_data, model_file_name)